In [0]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import joblib
import mlflow
import mlflow.pyfunc
import os

# Load dataset
df = pd.read_csv("/dbfs/FileStore/cars_latest.csv")

# Features and target
X = df[["year", "mileage"]]
y = df["price"]

# Train model
model = RandomForestRegressor()
model.fit(X, y)

# Save model locally
model_path = "car_price_model.pkl"
joblib.dump(model, model_path)

# Define custom MLflow model
class CarPriceModel(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        import joblib
        # Load from artifact path, not local path
        self.model = joblib.load(context.artifacts["model"])

    def predict(self, context, model_input):
        return self.model.predict(model_input)

# Log model and include the pickle file as artifact
with mlflow.start_run():
    mlflow.pyfunc.log_model(
        artifact_path="car_price_model",
        python_model=CarPriceModel(),
        artifacts={"model": model_path},  # <- attach it here
        input_example=pd.DataFrame({"year": [2010], "mileage": [150000]}),
        registered_model_name="car_price_model"
    )


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

/databricks/python/lib/python3.11/site-packages/mlflow/types/utils.py:394: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


Uploading artifacts:   0%|          | 0/11 [00:00<?, ?it/s]

Registered model 'car_price_model' already exists. Creating a new version of this model...


Uploading artifacts:   0%|          | 0/11 [00:00<?, ?it/s]

Created version '2' of model 'databrick_test_1231.default.car_price_model'.


Load & Predict using MLflow Model

In [0]:
import mlflow.pyfunc
import pandas as pd

# Prepare test input
input_df = pd.DataFrame({"year": [2019], "mileage": [10000]})

# Load the registered model
model_name = "databrick_test_1231.default.car_price_model"
model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/1")

# Predict
predictions = model.predict(input_df)
print(predictions)


[13015.25348214]
